# Convert h5 model to trt model

In [1]:
import sys 
import os

NOMEROFF_NET_DIR = "../../"

# setup result model dirs
SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/saved_model_options")
TRT_SAVED_MODEL_DIR  = os.path.join(NOMEROFF_NET_DIR, "./models/trt_saved_model_options")

sys.path.append(NOMEROFF_NET_DIR)

In [2]:
import tensorflow as tf
import matplotlib.image as mpimg

# dynamically grow the memory used on the GPU
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True 
sess = tf.compat.v1.Session(config=config)
set_session(sess)

from NomeroffNet import OptionsDetector

## load latest h5 model

In [3]:
optionsDetector = OptionsDetector()
optionsDetector.load("latest")

In [4]:
optionsDetector.MODEL.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 295, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 62, 293, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 31, 146, 32)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 29, 144, 64)  18496       max_pooling2d[0][0]              
_______________________________________________________________________________________

## convert to pb model

In [5]:
tf.saved_model.save(optionsDetector.MODEL, SAVED_MODEL_DIR)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../.././models/saved_model_options/assets


## convert pb model to trt model

In [6]:
params = tf.experimental.tensorrt.ConversionParams(
    minimum_segment_size=50,
    max_workspace_size_bytes=1 << 25,
    precision_mode  = "FP16")

converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir=SAVED_MODEL_DIR,
    conversion_params=params)
converter.convert()
converter.save(TRT_SAVED_MODEL_DIR)

INFO:tensorflow:Linked TensorRT version: (7, 1, 3)
INFO:tensorflow:Loaded TensorRT version: (7, 1, 3)
INFO:tensorflow:Could not find TRTEngineOp_0_0 in TF-TRT cache. This can happen if build() is not called, which means TensorRT engines will be built and cached at runtime.
INFO:tensorflow:Assets written to: ../.././models/trt_saved_model_options/assets


## run trt model

In [7]:
saved_model_loaded = tf.saved_model.load(TRT_SAVED_MODEL_DIR)

In [8]:
import matplotlib.image as mpimg
import numpy as np

img_path = '../../examples/crop_np_images/JJF509.png'
imgs = [mpimg.imread(img_path)]
Xs = []
for img in imgs:
    Xs.append(optionsDetector.normalize(img[:,:,:3]))

In [9]:
tensorX = tf.convert_to_tensor(Xs)

In [10]:
saved_model_loaded(tensorX)

[<tf.Tensor: shape=(1, 11), dtype=float32, numpy=
 array([[9.8546255e-01, 9.7715214e-04, 1.2638146e-03, 4.8207305e-03,
         2.8877663e-03, 6.1629072e-04, 5.6353782e-04, 1.7713576e-03,
         4.8201831e-04, 4.9773318e-04, 6.5710250e-04]], dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=
 array([[9.9431866e-01, 8.6607406e-04, 4.8148688e-03, 4.2924989e-07]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.98359156, 0.01524956, 0.00115885]], dtype=float32)>]